In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                url = 'https://www.rockauto.com/en/parts/' + input_.loc[a, 'Manufacturer'].replace('+', '%2B').replace(',', '%2C') + ',' + input_.loc[a, 'Part_Number'] + ',' + input_.loc[a, 'Type'] + ',' + input_.loc[a, 'Type_Code']
                
                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                no_parts_found = html.xpath('//span[@class="navlabellink nvoffset nnormal"]/text()')

                if len(no_parts_found) != 0 and no_parts_found[0].strip() == 'No Parts Found':
                    crawler_status = 'error'
                    break

                # = = = = = = = = = = = = = = =
                
                list_part = html.xpath('//tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-a-1") or contains(@class, "altrow-a-0"))]')
                
                # = = = = = = = = = = = = = = =
                
                list_json = [json.loads(part.xpath('./descendant::input[contains(@id, "listing_data_supplemental")]/@value')[0]) for part in list_part]

                # = = = = = = = = = = = = = = =
                
                index_part = -1
                for i in range(len(list_json)):
                    if list_json[i]['catalogname'].strip() == input_.loc[a, 'Manufacturer'] and list_json[i]['partnumber'].strip() == input_.loc[a, 'Part_Number']:
                        if index_part != -1:
                            continue
                        index_part = i

                # = = = = = = = = = = = = = = =
                
                list_note_1 = list_part[index_part].xpath('./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@class, "span-link-underline-remover")]/text()')
                note_1 = '' if len(list_note_1) == 0 else ';'.join([note_1.strip() for note_1 in list_note_1])

                # = = = = = = = = = = = = = = =
                
                list_note_2 = list_part[index_part].xpath('./descendant::span[contains(@title, "Please make sure")]/span/text()')
                note_2 = '' if len(list_note_2) == 0 else ';'.join([note_2.strip() for note_2 in list_note_2])

                # = = = = = = = = = = = = = = =

                src = list_part[index_part].xpath('./descendant::img[contains(@id, "inlineimg_thumb")]')
                src = '' if len(src) == 0 else 'https://www.rockauto.com' + src[0].xpath('./@src')[0].strip().replace('__ra_m.', '.')

                # = = = = = = = = = = = = = = =
                
                oe = list_part[index_part].xpath('./descendant::span[contains(@title, "Replaces these Alternate")]')
                oe = '' if len(oe) == 0 else oe[0].xpath('./text()')[0].strip().replace(', ', ';')
                oe = ';'.join(sorted(oe.split(';')))
                
                # = = = = = = = = = = = = = = =
                
                part_json = json.loads(list_part[index_part].xpath('./descendant::input[contains(@id, "listing_data_essential")]/@value')[0])
                part_key = part_json['partkey'].strip()

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'序号': int(input_.loc[a, 'No.']),
                                         'Manufacturer': input_.loc[a, 'Manufacturer'],
                                         'Category': input_.loc[a, 'Category'],
                                         'Type': input_.loc[a, 'Type'],
                                         'Type_Code': input_.loc[a, 'Type_Code'],
                                         'Part_Number': input_.loc[a, 'Part_Number'],
                                         'Vehcile': '',
                                         'OE': oe,
                                         'Note_1': note_1,
                                         'Note_2': note_2,
                                         'Pic': '',
                                         'Url': url,
                                         'Info': 'https://www.rockauto.com/en/moreinfo.php?pt=' + input_.loc[a, 'Type_Code'] + '&pk=' + part_key,
                                         'Src': src,
                                         'Part_Key': part_key,
                                         'JOIN_TCPK': input_.loc[a, 'Type_Code']+';'+part_key}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'No.': int(input_.loc[a, 'No.']),
                                     'Manufacturer': input_.loc[a, 'Manufacturer'],
                                     'Category': input_.loc[a, 'Category'],
                                     'Type': input_.loc[a, 'Type'],
                                     'Type_Code': input_.loc[a, 'Type_Code'],
                                     'Part_Number': input_.loc[a, 'Part_Number']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Part_Number'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：488

[ok] - 30250SX
[尝试次数：1] - [剩余数量：483] - [当前时间：09:46:00]

[ok] - 30430S
[尝试次数：2] - [剩余数量：482] - [当前时间：09:46:01]

[ok] - 30250S
[尝试次数：7] - [剩余数量：481] - [当前时间：09:46:03]

[ok] - 30460S
[尝试次数：3] - [剩余数量：480] - [当前时间：09:46:03]

[ok] - 30420S
[尝试次数：8] - [剩余数量：479] - [当前时间：09:46:04]

[ok] - 30460SA
[尝试次数：6] - [剩余数量：478] - [当前时间：09:46:04]

[ok] - 30460SB
[尝试次数：4] - [剩余数量：477] - [当前时间：09:46:05]

[ok] - 30470SX
[尝试次数：1] - [剩余数量：476] - [当前时间：09:46:05]

[ok] - 30490SAX
[尝试次数：3] - [剩余数量：475] - [当前时间：09:46:07]

[ok] - 30470S
[尝试次数：13] - [剩余数量：474] - [当前时间：09:46:09]

[ok] - 30490S
[尝试次数：20] - [剩余数量：473] - [当前时间：09:46:13]

[ok] - 30719SHWP
[尝试次数：10] - [剩余数量：472] - [当前时间：09:46:15]

[ok] - 31007S
[尝试次数：1] - [剩余数量：471] - [当前时间：09:46:15]

[ok] - 31035S
[尝试次数：4] - [剩余数量：470] - [当前时间：09:46:18]

[ok] - 30490SA
[尝试次数：34] - [剩余数量：469] - [当前时间：09:46:18]

[ok] - 31035SX
[尝试次数：5] - [剩余数量：468] - [当前时间：09:46:18]

[ok] - 30490SX
[尝试次数：34] - [剩余数量：467] - [当前时间：09:46:18]

[ok] - 31038SX
[尝试次数：4] - [剩余数量：466] - 

[ok] - 3359S
[尝试次数：5] - [剩余数量：336] - [当前时间：09:48:06]

[ok] - 3358S
[尝试次数：12] - [剩余数量：335] - [当前时间：09:48:07]

[ok] - 3359SC
[尝试次数：4] - [剩余数量：334] - [当前时间：09:48:07]

[ok] - 3359SB
[尝试次数：9] - [剩余数量：333] - [当前时间：09:48:07]

[ok] - 3361S
[尝试次数：8] - [剩余数量：332] - [当前时间：09:48:08]

[ok] - 3370S
[尝试次数：2] - [剩余数量：331] - [当前时间：09:48:09]

[ok] - 3365SA
[尝试次数：5] - [剩余数量：330] - [当前时间：09:48:09]

[ok] - 3374SA
[尝试次数：2] - [剩余数量：329] - [当前时间：09:48:12]

[ok] - 3372S
[尝试次数：9] - [剩余数量：328] - [当前时间：09:48:12]

[ok] - 3374S
[尝试次数：6] - [剩余数量：327] - [当前时间：09:48:12]

[ok] - 3370SA
[尝试次数：3] - [剩余数量：326] - [当前时间：09:48:13]

[ok] - 3376SA
[尝试次数：1] - [剩余数量：325] - [当前时间：09:48:15]

[ok] - 3378SA
[尝试次数：2] - [剩余数量：324] - [当前时间：09:48:15]

[ok] - 3376SB
[尝试次数：6] - [剩余数量：323] - [当前时间：09:48:16]

[ok] - 3376S
[尝试次数：8] - [剩余数量：322] - [当前时间：09:48:17]

[ok] - 3380SA
[尝试次数：1] - [剩余数量：321] - [当前时间：09:48:17]

[ok] - 3381AS
[尝试次数：1] - [剩余数量：320] - [当前时间：09:48:18]

[ok] - 3381A
[尝试次数：1] - [剩余数量：319] - [当前时间：09:48:18]

[ok] - 3380S
[尝试次

[ok] - 34202SB
[尝试次数：14] - [剩余数量：190] - [当前时间：09:50:07]

[ok] - 34207SX
[尝试次数：1] - [剩余数量：189] - [当前时间：09:50:09]

[ok] - 34207SAX
[尝试次数：7] - [剩余数量：188] - [当前时间：09:50:09]

[ok] - 34207SA
[尝试次数：11] - [剩余数量：187] - [当前时间：09:50:10]

[ok] - 34212S
[尝试次数：1] - [剩余数量：186] - [当前时间：09:50:11]

[ok] - 34207S
[尝试次数：19] - [剩余数量：185] - [当前时间：09:50:12]

[ok] - 34212SB
[尝试次数：1] - [剩余数量：184] - [当前时间：09:50:13]

[ok] - 34212SA
[尝试次数：4] - [剩余数量：183] - [当前时间：09:50:13]

[ok] - 34212SCX
[尝试次数：2] - [剩余数量：182] - [当前时间：09:50:16]

[ok] - 34214SA
[尝试次数：3] - [剩余数量：181] - [当前时间：09:50:17]

[ok] - 34220SX
[尝试次数：4] - [剩余数量：180] - [当前时间：09:50:19]

[ok] - 34201SXHWP
[尝试次数：21] - [剩余数量：179] - [当前时间：09:50:20]

[ok] - 34221SX
[尝试次数：2] - [剩余数量：178] - [当前时间：09:50:21]

[ok] - 34212SC
[尝试次数：4] - [剩余数量：177] - [当前时间：09:50:22]

[ok] - 34220SAX
[尝试次数：9] - [剩余数量：176] - [当前时间：09:50:23]

[ok] - 3491S
[尝试次数：3] - [剩余数量：175] - [当前时间：09:50:23]

[ok] - 3494SA
[尝试次数：1] - [剩余数量：174] - [当前时间：09:50:24]

[ok] - 3491SA
[尝试次数：1] - [剩余数量：173] - [当前时间

[ok] - 40217
[尝试次数：6] - [剩余数量：39] - [当前时间：09:52:28]

[ok] - 48510T9
[尝试次数：2] - [剩余数量：38] - [当前时间：09:52:29]

[ok] - 48530T9
[尝试次数：1] - [剩余数量：37] - [当前时间：09:52:29]

[ok] - 48500T9
[尝试次数：3] - [剩余数量：36] - [当前时间：09:52:29]

[ok] - 48520T9
[尝试次数：3] - [剩余数量：35] - [当前时间：09:52:30]

[ok] - 48560T9
[尝试次数：2] - [剩余数量：34] - [当前时间：09:52:30]

[ok] - 48551T9
[尝试次数：6] - [剩余数量：33] - [当前时间：09:52:31]

[ok] - 48562T9
[尝试次数：2] - [剩余数量：32] - [当前时间：09:52:33]

[ok] - 48563T9
[尝试次数：5] - [剩余数量：31] - [当前时间：09:52:34]

[ok] - 48564T9
[尝试次数：4] - [剩余数量：30] - [当前时间：09:52:35]

[ok] - 485636
[尝试次数：11] - [剩余数量：29] - [当前时间：09:52:35]

[ok] - 48561T9
[尝试次数：14] - [剩余数量：28] - [当前时间：09:52:36]

[ok] - 48567T9
[尝试次数：1] - [剩余数量：27] - [当前时间：09:52:36]

[ok] - 48566T9
[尝试次数：6] - [剩余数量：26] - [当前时间：09:52:38]

[ok] - 485683
[尝试次数：5] - [剩余数量：25] - [当前时间：09:52:39]

[ok] - 485699
[尝试次数：7] - [剩余数量：24] - [当前时间：09:52:40]

[ok] - 48565T9
[尝试次数：13] - [剩余数量：23] - [当前时间：09:52:40]

[ok] - 48570T9
[尝试次数：5] - [剩余数量：22] - [当前时间：09:52:40]

[ok] - 48571